In [1]:
import pandas as pd
from langchain_ollama import ChatOllama

excel_file = 'XML_DATA.xlsx'
df = pd.read_excel(excel_file, engine='openpyxl')
text_content = df.to_string()
text_content

'                           RULE_NAME                  RULE_TYPE              OUTPUT_REFERENCE_NAME\n0  SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR      SECONDARY_MEASUREMENT   SM_PL_VAR_COMP_LOSS_RATIO_FACTOR\n1                CR_VAR_COMP_PREMIUM  DIRECT_TRANSACTION_CREDIT                 C_VAR_COMP_PREMIUM\n2   SM_PL_VAR_COMP_LOSS_RATIO_FACTOR      SECONDARY_MEASUREMENT  SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR\n3                 C_VAR_COMP_PREMIUM  DIRECT_TRANSACTION_CREDIT                CR_VAR_COMP_PREMIUM'

In [2]:
df = df[['RULE_TYPE', 'RULE_NAME', 'OUTPUT_REFERENCE_NAME']]
df

,RULE_TYPE,RULE_NAME,OUTPUT_REFERENCE_NAME
0,SECONDARY_MEASUREMENT,SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR,SM_PL_VAR_COMP_LOSS_RATIO_FACTOR
1,DIRECT_TRANSACTION_CREDIT,CR_VAR_COMP_PREMIUM,C_VAR_COMP_PREMIUM
2,SECONDARY_MEASUREMENT,SM_PL_VAR_COMP_LOSS_RATIO_FACTOR,SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR
3,DIRECT_TRANSACTION_CREDIT,C_VAR_COMP_PREMIUM,CR_VAR_COMP_PREMIUM


# Final Approach

In [4]:
import pandas as pd
import ollama

# Step 1: Sample DataFrame
# data = {
#     "RULE_TYPE": ["SECONDARY_MEASUREMENT", "PRIMARY_MEASUREMENT"],
#     "RULE_NAME": ["SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR", "WRONG_NAME"],
#     "OUTPUT_REFERENCE_NAME": ["SM_PL_VAR_COMP_LOSS_RATIO_FACTOR", "XYZ_OUTPUT"]
# }
# df = pd.DataFrame(data)

# Step 2: Few-shot examples
few_shots = [
    {
        "RULE_TYPE": "SECONDARY_MEASUREMENT",
        "RULE_NAME": "SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR",
        "OUTPUT_REFERENCE_NAME": "SM_PL_VAR_COMP_LOSS_RATIO_FACTOR",
        "result": "VALID"
    },
    {
        "RULE_TYPE": "SECONDARY_MEASUREMENT",
        "RULE_NAME": "WRONG_NAME",
        "OUTPUT_REFERENCE_NAME": "SM_PL_VAR_COMP_LOSS_RATIO_FACTOR",
        "result": "Not Matching"
    },
    {
        "RULE_TYPE": "SECONDARY_MEASUREMENT",
        "RULE_NAME": "SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR",
        "OUTPUT_REFERENCE_NAME": "WRONG_OUTPUT",
        "result": "Not Matching"
    },
    {
        "RULE_TYPE": "DIRECT_TRANSACTION_CREDIT	",
        "RULE_NAME": "CR_VAR_COMP_PREMIUM",
        "OUTPUT_REFERENCE_NAME": "C_VAR_COMP_PREMIUM",
        "result": "VALID"
    },
    {
        "RULE_TYPE": "DIRECT_TRANSACTION_CREDIT	",
        "RULE_NAME": "WRONG_NAME",
        "OUTPUT_REFERENCE_NAME": "WRONG_NAME",
        "result": "Not Matching"
    }
    # {
    #     "RULE_TYPE": "DIRECT_TRANSACTION_CREDIT	",
    #     "RULE_NAME": "CR_VAR_COMP_PREMIUM",
    #     "OUTPUT_REFERENCE_NAME": "WRONG_NAME",
    #     "result": "INVALID"
    # }
]

# Step 3: Build prompt
def build_prompt(row, few_shots):
    prompt = "You are a smart data validator.\n"
    prompt += "Rule: If RULE_TYPE is 'SECONDARY_MEASUREMENT', then RULE_NAME must be 'SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR' and OUTPUT_REFERENCE_NAME must be 'SM_PL_VAR_COMP_LOSS_RATIO_FACTOR' on the other hand  RULE_TYPE is 'DIRECT_TRANSACTION_CREDIT	', then RULE_NAME must be 'CR_VAR_COMP_PREMIUM' and OUTPUT_REFERENCE_NAME must be 'C_VAR_COMP_PREMIUM'.\n\n"
    prompt += "Here are some examples:\n"
    for shot in few_shots:
        example = f"RULE_TYPE: {shot['RULE_TYPE']}, RULE_NAME: {shot['RULE_NAME']}, OUTPUT_REFERENCE_NAME: {shot['OUTPUT_REFERENCE_NAME']} -> {shot['result']}\n"
        prompt += example
    prompt += "\nNow evaluate the following only answer whether its is VALID or Not Matching:\n"
    prompt += f"RULE_TYPE: {row['RULE_TYPE']}, RULE_NAME: {row['RULE_NAME']}, OUTPUT_REFERENCE_NAME: {row['OUTPUT_REFERENCE_NAME']}\n"
    prompt += "Result:"
    return prompt

# Step 4: Call local LLaMA model using Ollama
def validate_row_with_llama(row):
    prompt = build_prompt(row, few_shots)
    response = ollama.chat(
        model="llama3",
        messages=[
            {"role": "system", "content": "You are a smart data validator."},
            {"role": "user", "content": prompt}
        ],
        options={
            'temperature': 0
            }
        # temperature=0  # Ensure deterministic validation
    )
    return response['message']['content'].strip()

# Step 5: Apply to DataFrame
df["LLM_VALIDATION"] = df.apply(validate_row_with_llama, axis=1)

# Print the results
print(df)

                   RULE_TYPE                          RULE_NAME  \
0      SECONDARY_MEASUREMENT  SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR   
1  DIRECT_TRANSACTION_CREDIT                CR_VAR_COMP_PREMIUM   
2      SECONDARY_MEASUREMENT   SM_PL_VAR_COMP_LOSS_RATIO_FACTOR   
3  DIRECT_TRANSACTION_CREDIT                 C_VAR_COMP_PREMIUM   

               OUTPUT_REFERENCE_NAME LLM_VALIDATION  
0   SM_PL_VAR_COMP_LOSS_RATIO_FACTOR          VALID  
1                 C_VAR_COMP_PREMIUM          VALID  
2  SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR   Not Matching  
3                CR_VAR_COMP_PREMIUM   Not Matching  


In [5]:
df.iloc[3,3]

'Not Matching'

In [6]:
df

,RULE_TYPE,RULE_NAME,OUTPUT_REFERENCE_NAME,LLM_VALIDATION
0,SECONDARY_MEASUREMENT,SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR,SM_PL_VAR_COMP_LOSS_RATIO_FACTOR,VALID
1,DIRECT_TRANSACTION_CREDIT,CR_VAR_COMP_PREMIUM,C_VAR_COMP_PREMIUM,VALID
2,SECONDARY_MEASUREMENT,SM_PL_VAR_COMP_LOSS_RATIO_FACTOR,SMR_PL_VAR_COMP_LOSS_RATIO_FACTOR,Not Matching
3,DIRECT_TRANSACTION_CREDIT,C_VAR_COMP_PREMIUM,CR_VAR_COMP_PREMIUM,Not Matching


In [12]:
df.to_excel('sample_naming_op.xlsx')